In [1]:
import pandas as pd
import numpy as np
from keras.layers import Dense, LSTM, SimpleRNN, Embedding
from keras.models import Sequential
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [2]:
raw_data = pd.read_csv('train.csv', encoding='latin-1')
raw_test_data = pd.read_csv('test.csv', encoding='latin-1')

print(raw_data.shape)
print(raw_data.columns)

(29000, 2)
Index(['Label', 'Message'], dtype='object')


In [3]:
raw_test_data.columns

Index(['Label', 'Message'], dtype='object')

In [4]:
raw_data.Message

0                       oh how abt 2 days before Christmas
1        Welcome to OVATION HOLD R.No. 184, 114, 395, 3...
2        Thank you for using your ICICI bank CREDITcard...
3        schedule a meeting with the entire team in the...
4                                      Tommy is my brother
5        OTP is 817453 for the txn of INR 8262.00 at SP...
6                         the meeting is scheduled by john
7        Dear customer, We wish you a Merry Christmas. ...
8        Delivered: Your package withPawzone Red 1.25 i...
9        The PNR for your Air India Flt 7I115 for PGH-B...
10       Bimal Auto Agency : Service of your car KA52C8...
11       Appointment with Dr Clayton in Pune on 2011-08...
12       Maha Veer Auto Agency : Service of your car KA...
13       Dear AirAsia Customer, flight 5Q658 from RJA s...
14       Dear Guest, Thanks for choosing Forlini's Rest...
15                               I will indeed! What time?
16       Dear Guest, Thanks for choosing 2nd Avenue Del.

In [5]:
char_to_idx = {ch: i for (i, ch) in enumerate(sorted(list(set(raw_data.Message))))}

print(len(char_to_idx))

28926


In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(raw_data.Message)

train_x = tokenizer.texts_to_sequences(raw_data.Message)
train_x = pad_sequences(train_x, maxlen=50)
test_x = tokenizer.texts_to_sequences(raw_test_data.Message)
test_x = pad_sequences(test_x, maxlen=50)

print(train_x.shape, test_x.shape)

unique_labels = list(raw_data.Label.unique())
train_y = np.array([unique_labels.index(i) for i in raw_data.Label])
train_y = to_categorical(train_y)
test_y = np.array([unique_labels.index(i) for i in raw_test_data.Label])
test_y = to_categorical(test_y)

(29000, 50) (1000, 50)


## Create a model which gives above 99% accuracy on test data.

In [7]:
train_x[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  651,
        474, 1026,   98,  115,  346,  488])

In [13]:
train_x = train_x.reshape(29000,50,1)

In [35]:
from keras.models import Sequential
from keras.layers.recurrent import LSTM, SimpleRNN
from keras.layers import Dense,Activation

model = Sequential()
model.add(LSTM(50, input_shape=(50, 1), return_sequences=False))
model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


In [36]:
model.fit(train_x,train_y)

Epoch 1/10
29000/29000 [==============================] - 37s - loss: 0.2697 - acc: 0.8941    
Epoch 2/10
29000/29000 [==============================] - 36s - loss: 0.1035 - acc: 0.9607    
Epoch 3/10
29000/29000 [==============================] - 36s - loss: 0.0775 - acc: 0.9741    
Epoch 4/10
29000/29000 [==============================] - 36s - loss: 0.0739 - acc: 0.9783    
Epoch 5/10
29000/29000 [==============================] - 36s - loss: 0.0631 - acc: 0.9807    
Epoch 6/10
29000/29000 [==============================] - 54s - loss: 0.0541 - acc: 0.9830    
Epoch 7/10
29000/29000 [==============================] - 62s - loss: 0.0506 - acc: 0.9835    
Epoch 8/10
29000/29000 [==============================] - 60s - loss: 0.0489 - acc: 0.9837    - ETA: 1s - loss
Epoch 9/10
29000/29000 [==============================] - 61s - loss: 0.0426 - acc: 0.9860    - ETA
Epoch 10/10
29000/29000 [==============================] - 60s - loss: 0.0424 - acc: 0.9871    


In [37]:
test_x = test_x.reshape(1000,50,1)
predict = model.predict_classes(test_x) 

1000/1000 [==============================] - 0s     


In [38]:
predict

array([0, 1, 1, 2, 1, 1, 1, 2, 0, 0, 0, 0, 2, 1, 0, 0, 1, 0, 1, 1, 2, 0, 2,
       2, 1, 1, 1, 1, 1, 2, 0, 1, 1, 1, 2, 1, 1, 0, 1, 0, 0, 2, 1, 1, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 2, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 2, 1, 2, 1, 0, 2, 1, 2, 1, 2, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 2, 1,
       1, 0, 1, 2, 1, 1, 1, 2, 1, 2, 0, 0, 1, 2, 0, 1, 0, 0, 1, 2, 1, 1, 2,
       0, 0, 0, 2, 0, 2, 1, 0, 1, 1, 0, 2, 1, 1, 0, 2, 1, 1, 2, 0, 1, 0, 0,
       1, 1, 0, 1, 1, 0, 0, 1, 2, 0, 0, 1, 1, 1, 0, 1, 1, 2, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 1, 2, 1, 0, 2, 0, 0, 2, 0, 1, 1, 2, 1, 2, 2, 1, 1, 0, 1,
       1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 2, 0, 0, 0, 2, 0, 0, 0, 1, 2, 0, 1,
       1, 1, 0, 2, 2, 1, 2, 1, 0, 1, 0, 1, 2, 0, 1, 1, 0, 2, 0, 0, 1, 1, 0,
       1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 2, 0, 1, 2, 1, 2, 1, 0,
       0, 1, 2, 2, 0, 1, 0, 2, 1, 1, 0, 0, 2, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0,
       2, 0, 2, 2, 1, 1, 1, 2, 0, 0, 1, 2, 1, 1, 1, 1, 1, 0, 2, 1, 1, 2, 0,
       0, 0,

In [39]:
model.evaluate(test_x,test_y)[1]

 992/1000 [============================>.] - ETA: 0s

0.98599999999999999